In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Funciones

In [2]:
def getElementsByTag(driver, xpath, tag):
    element = driver.find_element_by_xpath(xpath).find_elements_by_tag_name(tag)
    return element

In [9]:
def get_table(driver):
    """
    Parsear tabla con beautifulSoup
    Returns: dataframe
    """
    html = driver.page_source
    sopa = BeautifulSoup(html, 'html.parser')
    tds = sopa.find_all("td", {"style": ["width:7%;", "width:15%;", "width:9%;", "width:12%;", "width:2%;"]})
    titles = sopa.find_all("th")
    lista_ds = []
    i = 0
    while i < len(tds):
        d = {}
        d[titles[1].text] = tds[i].text
        d[titles[2].text] = tds[i + 1].text
        d[titles[3].text] = tds[i + 2].text
        d[titles[4].text] = tds[i + 3].text
        d[titles[5].text] = tds[i + 4].text
        d[titles[6].text] = tds[i + 5].text
        d[titles[7].text] = tds[i + 6].text
        lista_ds.append(d)
        i += 7
    df = pd.DataFrame.from_dict(lista_ds)
    return df

In [10]:
def get_pdf_links(df, driver):
    """
    Abrir link para cada pdf, extraerlo y ponerlo en el df
    Parameter: Dataframe
    Returns: Dataframe
    """
    rowsPath = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody'
    btnCerrarVisor = '//*[@id="modalPDF"]/div/div/div[3]/button'    
    visorPDF = '//*[@id="ContentPlaceHolder1_framePDF"]'
    pdf_links = []
    j = 0    
    while (j < len(df['Sentencia'])):             
        rows = getElementsByTag(driver, rowsPath, "tr")        
        cols = rows[j].find_elements_by_tag_name("td")       

        # nCol = len(cols)
        #Abrir pdf para obtener link
        try:                    
            element = cols[-1]
            #driver.execute_script("arguments[0].click;", element)
            element.click()
        except Exception as e:
            print(str(e))
            print(f"PDF not clickable")                    
        #Esperamos a que cargue
        time.sleep(5)        
        pdf_link = driver.find_element_by_xpath(visorPDF).get_attribute('src')
        pdf_links.append(pdf_link)
        #Cerrar el visor
        time.sleep(5)
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        #button_close = driver.find_element_by_xpath(btnCerrarVisor)
        #driver.execute_script("arguments[0].scrollIntoView(true);", button_close)
        #button_close.click()
        # driver.execute_script("arguments[0].click;",button_close)
        j += 1                        
        #Boton descargar
        #//*[@id="download"]  
        #print("termina "+str(j))  
    time.sleep(4)
    df["Sentencia"] = pdf_links
    return df   

## Deprecated (o sea, no sirve por ahora)

# Pipeline

In [5]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.pjenl.gob.mx/SentenciasPublicas/Modulos/Penales.aspx")

In [6]:
dfs_list = []
num = 5
pagina = 2
while pagina < 16:
    if num > 11 and pagina == 1:
        print(f"{pagina} extracted!")
        num = 3
        pagina += 1
        final_df = pd.concat(dfs_list, ignore_index=True)
        final_df.to_csv(f"sentencias_penales_pag{pagina}.csv", 
        index=False, encoding='utf-8-sig')
        dfs_list = [] 

    elif num > 12 and pagina > 1:
        print(f"{pagina} extracted!")
        num = 3
        pagina += 1
        final_df = pd.concat(dfs_list, ignore_index=True)
        final_df.to_csv(f"sentencias_penales_pag{pagina}.csv", 
        index=False, encoding='utf-8-sig')
        dfs_list = []

    df = get_table(driver)
    df = get_pdf_links(df, driver)
    dfs_list.append(df)
    next_xpath = f'//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[{num}]/a'
    try:
        next_button = driver.find_element_by_xpath(next_xpath)
        next_button.click()
        print("next clicked!")
    except Exception as e:
        print(str(e))
        next_xpath = f'//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[22]/td/table/tbody/tr/td[{num}]/a'
        next_button = driver.find_element_by_xpath(next_xpath)
        next_button.click()
        print("alternative next clicked!")

    num = num + 1
    time.sleep(45)
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME) 

ate element: {"method":"xpath","selector":"//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[11]/a"}
  (Session info: chrome=90.0.4430.93)

alternative next clicked!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[12]/a"}
  (Session info: chrome=90.0.4430.93)

alternative next clicked!
7 extracted!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[3]/a"}
  (Session info: chrome=90.0.4430.93)

alternative next clicked!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[4]/a"}
  (Session info: chrome=90.0.4430.93)

alternative next clicked!
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ContentPlaceHolder1_g

### Sigamos:

In [8]:
print([num, pagina])

[4, 16]


In [14]:
dfs_list = []
num = 12
pagina = 16
while pagina < 30:
    if num > 11 and pagina == 1:
        print(f"{pagina} extracted!")
        num = 3
        pagina += 1
        final_df = pd.concat(dfs_list, ignore_index=True)
        final_df.to_csv(f"sentencias_penales_pag{pagina}.csv", 
        index=False, encoding='utf-8-sig')
        dfs_list = [] 

    elif num > 12 and pagina > 1:
        print(f"{pagina} extracted!")
        num = 3
        pagina += 1
        final_df = pd.concat(dfs_list, ignore_index=True)
        final_df.to_csv(f"sentencias_penales_pag{pagina}.csv", 
        index=False, encoding='utf-8-sig')
        dfs_list = []

    df = get_table(driver)
    df = get_pdf_links(df, driver)
    dfs_list.append(df)
    next_xpath = f'//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[{num}]/a'
    try:
        next_button = driver.find_element_by_xpath(next_xpath)
        next_button.click()
        print("next clicked!")
    except Exception as e:
        print(str(e))
        next_xpath = f'//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[22]/td/table/tbody/tr/td[{num}]/a'
        next_button = driver.find_element_by_xpath(next_xpath)
        next_button.click()
        print("alternative next clicked!")

    num = num + 1
    time.sleep(45)
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME) 

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[12]/a"}
  (Session info: chrome=90.0.4430.93)

alternative next clicked!
16 extracted!
list index out of range
PDF not clickable
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[3]/a"}
  (Session info: chrome=90.0.4430.93)

alternative next clicked!
list index out of range
PDF not clickable
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr/td[4]/a"}
  (Session info: chrome=90.0.4430.93)

alternative next clicked!
list index out of range
PDF not clickable


KeyboardInterrupt: 

In [15]:
pagina

17